In [1]:
import pandas as pd

### Load Data

In [2]:
london_raw = pd.read_csv('london.csv')

In [3]:
station_info = pd.read_csv('london_stations.csv')

In [4]:
station_info.head()

,station_id,station_name,longitude,latitude
0,1,"River Street, Clerkenwell",-0.109971,51.5292
1,2,"Phillimore Gardens, Kensington",-0.197574,51.4996
2,3,"Christopher Street, Liverpool Street",-0.084606,51.5213
3,4,"St. Chad's Street, King's Cross",-0.120974,51.5301
4,5,"Sedding Street, Sloane Square",-0.156876,51.4931


### Preprocessing

##### 1. Remove NAs

In [5]:
london = london_raw.dropna(axis=0)

In [6]:
print("raw   data :", len(london_raw))
print("na rm data :", len(london))

raw   data : 38215560
na rm data : 38147278


##### 2. Convert data type

In [7]:
# convert type to integer of `start_station_id` & `end_station_id`
london['start_station_id'] = london['start_station_id'].astype('int64')
london['end_station_id'] = london['end_station_id'].astype('int64')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### 3. Filter stations which do not appear in the `station_info`

In [8]:
station_list = set(london['start_station_id'].unique()) & set(station_info['station_id'].unique()) & set(london['end_station_id'].unique())

In [9]:
len(london['start_station_id'].unique())

807

In [10]:
len(london['end_station_id'].unique())

808

In [11]:
len(station_list)

801

In [12]:
start_id_unk = set.difference(set(london['start_station_id']), station_list)

In [13]:
end_id_unk = set.difference(set(london['end_station_id']), station_list)

In [14]:
station_id_unk = list(start_id_unk | end_id_unk)
station_id_unk

[391, 304, 241, 434, 823, 825, 346]

In [15]:
london = london[~london.start_station_id.isin(station_id_unk)]

In [16]:
london = london[~london.end_station_id.isin(station_id_unk)]

### Create  co-occurance matrix

#### 1. before COVID / after COVID

In [17]:
london.iloc(0)[0]

rental_id                                                 61343322
duration                                                        60
bike_id                                                      12871
end_rental_date_time                           2016-12-28 00:01:00
end_station_id                                                 660
end_station_name          West Kensington Station, West Kensington
start_rental_date_time                         2016-12-28 00:00:00
start_station_id                                               633
start_station_name             Vereker Road North, West Kensington
Name: 0, dtype: object

In [18]:
# london.loc[0,'end_rental_date_time'] < '2016-12-28 24:00:00'
london_before_covid = london.query("start_rental_date_time >= '2019-01-01' and start_rental_date_time < '2019-09-01'")
london_after_covid = london.query("start_rental_date_time >= '2020-01-01' and start_rental_date_time < '2020-09-01'")

In [19]:
london_before_covid.head()

,rental_id,duration,bike_id,end_rental_date_time,end_station_id,end_station_name,start_rental_date_time,start_station_id,start_station_name
20871718,83255489,1620.0,13492.0,2019-01-01 00:27:00,228,"St. James's Square, St. James's",2019-01-01 00:00:00,228,"St. James's Square, St. James's"
20871719,83255490,480.0,13770.0,2019-01-01 00:08:00,212,"Campden Hill Road, Notting Hill",2019-01-01 00:00:00,155,"Lexham Gardens, Kensington"
20871720,83255488,1680.0,8652.0,2019-01-01 00:28:00,228,"St. James's Square, St. James's",2019-01-01 00:00:00,228,"St. James's Square, St. James's"
20871721,83255494,6000.0,5063.0,2019-01-01 01:41:00,188,"Nutford Place, Marylebone",2019-01-01 00:01:00,44,"Bruton Street, Mayfair"
20871722,83255497,840.0,6116.0,2019-01-01 00:15:00,400,"George Street, Marylebone",2019-01-01 00:01:00,751,"Newton Street, Covent Garden"


In [20]:
co_matrix_bc = pd.crosstab(london_before_covid['start_station_id'], london_before_covid['end_station_id'])
co_matrix_ac = pd.crosstab(london_after_covid['start_station_id'], london_after_covid['end_station_id'])

In [21]:
co_matrix_bc.to_csv("co_matrix_bc.csv")

In [22]:
co_matrix_ac.to_csv("co_matrix_ac.csv")

#### 2. Time : 7-10 AM / 13-16 PM / 17-20 PM (start_rental_date_time)

In [28]:
london_7_10 = london[pd.to_datetime((london['start_rental_date_time'])).dt.strftime('%H:%M:%S').between('07:00:00','10:00:00')]

In [32]:
london_13_16 = london[pd.to_datetime((london['start_rental_date_time'])).dt.strftime('%H:%M:%S').between('13:00:00','16:00:00')]

In [30]:
london_17_20 = london[pd.to_datetime((london['start_rental_date_time'])).dt.strftime('%H:%M:%S').between('17:00:00','20:00:00')]

In [33]:
co_matrix_7_10 = pd.crosstab(london_7_10['start_station_id'], london_7_10['end_station_id'])
co_matrix_13_16 = pd.crosstab(london_13_16['start_station_id'], london_13_16['end_station_id'])
co_matrix_17_20 = pd.crosstab(london_17_20['start_station_id'], london_17_20['end_station_id'])

In [34]:
co_matrix_7_10.to_csv("co_matrix_7_10.csv")
co_matrix_13_16.to_csv("co_matrix_13_16.csv")
co_matrix_17_20.to_csv("co_matrix_17_20.csv")

In [36]:
print('# 07-10 :', len(london_7_10))
print('# 13-16 :', len(london_13_16))
print('# 17-20 :', len(london_17_20))

# 07-10 : 7968419
# 13-16 : 6621299
# 17-20 : 10058478


#### 3. Duration : ~ 10m / ~ 30m / ~ 60m / ~ 180m / 180m ~

In [48]:
london_10m = london.query("duration < 60 * 10")

In [52]:
london_30m = london.query("duration >= 60 * 10 and duration < 60 * 30")

In [53]:
london_60m = london.query("duration >= 60 *30 and duration < 60 * 60")

In [54]:
london_180m = london.query("duration >= 60 *60 and duration < 60 * 180")

In [61]:
london_over_180m = london.query("duration >= 60 * 180")

In [65]:
print('# 00m  <= minutes < 10m  :', len(london_10m))
print('# 10m  <= minutes < 30m  :', len(london_30m))
print('# 30m  <= minutes < 60m  :', len(london_60m))
print('# 60m  <= minutes < 180m :', len(london_180m))
print('# 180m <= minutes        :', len(london_over_180m))

# 00m  <= minutes < 10m  : 11505774
# 10m  <= minutes < 30m  : 21874962
# 30m  <= minutes < 60m  : 3150085
# 60m  <= minutes < 180m : 1353686
# 180m <= minutes        : 253423


In [66]:
london_10m.to_csv("london_10m.csv")
london_30m.to_csv("london_30m.csv")
london_60m.to_csv("london_60m.csv")
london_180m.to_csv("london_180m.csv")
london_over_180m.to_csv("london_over_180m.csv")